
# **Image Colorization With GANs**

GANs are the state-of-the-art machine learning models which can generate new data instances from existing ones. They use a very interesting technique, inspired from the Game Theory, to generate realistic samples.

In this notebook, we'll use GANs to colorize a grayscale ( B/W ) image. In addition to that, our generator model will have a structure similar to that of a UNet i.e the one with skip connections.



## **1. Downloading and Processing the data**

A dataset of RGB images to train the GAN model whose images consists of various scenes/places.

* Download the dataset on your machine from [here](https://drive.google.com/file/d/1sQ5C8HiKVr2Edp3ojLLNauwRbOLfVn2q/view?usp=sharing).
* Upload the downloaded `.zip` file here on Colab.







In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



We'll now parse the images ( RGB images to be precise ) one by one, and transform each one to a grayscale image using PIL's `.convert( 'L' )` method. So our dataset will have samples of $( \ grayscale \ image \ , \ RGB \ image \ )$

We used only a part of our dataset, determined by `dataset_split` , as Colab's computational power would cease on providing a large number of images.


In [2]:
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from matplotlib import image
from matplotlib import pyplot as plt
import os
from tensorflow import keras

batch_size = 64

img_size = 128

dataset_split = 600

master_dir = '/content/drive/MyDrive/Mestrado/2024_2/Topicos_especiais_IA_ThiagoPX/NormalAndMonet'
x = []
y = []
for image_file in os.listdir( master_dir )[ 0 : dataset_split ]:
    rgb_image = Image.open( os.path.join( master_dir , image_file ) ).resize( ( img_size , img_size ) )

    rgb_img_array = (np.asarray( rgb_image ) ) / 255
    gray_image = rgb_image.convert( 'L' )

    gray_img_array = ( np.asarray( gray_image ).reshape( ( img_size , img_size , 1 ) ) ) / 255

    x.append( gray_img_array )
    y.append( rgb_img_array )

train_x, test_x, train_y, test_y = train_test_split( np.array(x) , np.array(y) , test_size=0.1 )

dataset = tf.data.Dataset.from_tensor_slices( ( train_x , train_y ) )
dataset = dataset.batch( batch_size )


In [ ]:
import tensorflow as tf
import numpy as np
import os

# Parameters
batch_size = 64
img_size = 128
dataset_split = 600
master_dir = '/content/drive/MyDrive/Mestrado/2024_2/Topicos_especiais_IA_ThiagoPX/NormalAndMonet'

# Image loading and preprocessing
def load_and_preprocess_image(image_path):
    # Load RGB image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = img / 255.0  # Normalize to [0, 1]

    # Convert to grayscale
    gray_img = tf.image.rgb_to_grayscale(img)
    return gray_img, img

# Create dataset
image_files = [os.path.join(master_dir, fname) for fname in os.listdir(master_dir)[:dataset_split]]
dataset = tf.data.Dataset.from_tensor_slices(image_files)

# Map the preprocessing function
dataset = dataset.map(lambda x: load_and_preprocess_image(x))

# Split dataset into train and test sets
train_size = int(0.9 * len(image_files))  # 90% for training
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# Batch the datasets
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# Prefetch for performance optimization
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

# Print to check the datasets
print(f"Train Dataset: {len(list(train_dataset))} batches")
print(f"Test Dataset: {len(list(test_dataset))} batches")



## **2. The GAN**

In this section, we'll create our GAN model step-by-step with Keras. First, we'll implement the generator then the discriminator and finally the loss functions required by both of them.



### **A. Generator**

Our generator ( represented as $G$ ) will take in grayscale image $x$ and produce a RGB image $G( x )$. Note, $x$ will be a tensor of shape $( \ batch \ size \ , \ 120 \ , \ 120 \ , \ 1 \ )$ and the output $G(x)$ will have a shape $( \ batch \ size \ , \ 120 \ , \ 120 \ , \ 3 \ )$

* Our generator will have a encoder-decoder structure, similar to the UNet architecture. Additionally, we use Dilated convolutions to have a larger receptive field.

* We introduce skip connections in our model so as to have better flow of information from the encoder to the decoder.


In [3]:
def get_generator_model():
    inputs = tf.keras.layers.Input(shape=(img_size, img_size, 1))

    # Encoder com Dropout
    conv1 = tf.keras.layers.Conv2D(16, kernel_size=(5, 5), strides=1)(inputs)
    conv1 = tf.keras.layers.LeakyReLU()(conv1)
    conv1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1)(conv1)
    conv1 = tf.keras.layers.LeakyReLU()(conv1)
    conv1 = tf.keras.layers.Dropout(0.3)(conv1)
    conv1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1)(conv1)
    conv1 = tf.keras.layers.LeakyReLU()(conv1)

    conv2 = tf.keras.layers.Conv2D(32, kernel_size=(5, 5), strides=1)(conv1)
    conv2 = tf.keras.layers.LeakyReLU()(conv2)
    conv2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1)(conv2)
    conv2 = tf.keras.layers.LeakyReLU()(conv2)
    conv2 = tf.keras.layers.Dropout(0.3)(conv2)
    conv2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1)(conv2)
    conv2 = tf.keras.layers.LeakyReLU()(conv2)

    conv3 = tf.keras.layers.Conv2D(64, kernel_size=(5, 5), strides=1)(conv2)
    conv3 = tf.keras.layers.LeakyReLU()(conv3)
    conv3 = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1)(conv3)
    conv3 = tf.keras.layers.LeakyReLU()(conv3)
    conv3 = tf.keras.layers.Dropout(0.3)(conv3)
    conv3 = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1)(conv3)
    conv3 = tf.keras.layers.LeakyReLU()(conv3)

    # Bottleneck
    bottleneck = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1, activation='tanh', padding='same')(conv3)

    # Decoder com Dropout
    concat_1 = tf.keras.layers.Concatenate()([bottleneck, conv3])
    conv_up_3 = tf.keras.layers.Conv2DTranspose(128, kernel_size=(3, 3), strides=1, activation='relu')(concat_1)
    conv_up_3 = tf.keras.layers.Conv2DTranspose(128, kernel_size=(3, 3), strides=1, activation='relu')(conv_up_3)
    conv_up_3 = tf.keras.layers.Dropout(0.3)(conv_up_3)
    conv_up_3 = tf.keras.layers.Conv2DTranspose(64, kernel_size=(5, 5), strides=1, activation='relu')(conv_up_3)

    concat_2 = tf.keras.layers.Concatenate()([conv_up_3, conv2])
    conv_up_2 = tf.keras.layers.Conv2DTranspose(64, kernel_size=(3, 3), strides=1, activation='relu')(concat_2)
    conv_up_2 = tf.keras.layers.Conv2DTranspose(64, kernel_size=(3, 3), strides=1, activation='relu')(conv_up_2)
    conv_up_2 = tf.keras.layers.Dropout(0.3)(conv_up_2)
    conv_up_2 = tf.keras.layers.Conv2DTranspose(32, kernel_size=(5, 5), strides=1, activation='relu')(conv_up_2)

    concat_3 = tf.keras.layers.Concatenate()([conv_up_2, conv1])
    conv_up_1 = tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=1, activation='relu')(concat_3)
    conv_up_1 = tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=1, activation='relu')(conv_up_1)
    conv_up_1 = tf.keras.layers.Conv2DTranspose(3, kernel_size=(5, 5), strides=1, activation='relu')(conv_up_1)

    model = tf.keras.models.Model(inputs, conv_up_1)
    return model



### **B. Discriminator**

The discriminator model, represented as $D$, will take in the *real image* $y$ ( from the training data ) and the *generated image* $G(x)$ ( from the generator ) to output two probabilities.

* We train the discriminator in such a manner that is able to differentiate the *real images* and the generated *images*. So, we train the model such that $y$ produces a output of $1.0$ and $G(x)$ produces an output of $0.0$.
* Note that instead of using hard labels like $1.0$ and $0.0$, we use soft labels which are close to 1 and 0. So for a hard label of $1.0$, the soft label will be $(1 - \epsilon)$ where $\epsilon$ is picked uniformly from $( 0 , 0.1 ]$


In [4]:
def get_discriminator_model():
    layers = [
        tf.keras.layers.Conv2D(32, kernel_size=(7, 7), strides=1, activation='relu', input_shape=(img_size, img_size, 3)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(32, kernel_size=(7, 7), strides=1, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, kernel_size=(5, 5), strides=1, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(64, kernel_size=(5, 5), strides=1, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ]
    model = tf.keras.models.Sequential(layers)
    return model



### **C. Loss Functions**

We'll now implement the loss functions for our GAN model. As you might know that we have two loss functions, one for the generator and another for the discriminator.

* For our generator, we'll use the L2/MSE loss function.
* For optimization, we use the Adam optimizer with a learning rate of 0.0005



In [5]:
import tensorflow as tf

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
mse = tf.keras.losses.MeanSquaredError()

def discriminator_loss(real_output, fake_output):

    real_labels = tf.ones_like(real_output) - tf.random.uniform(shape=real_output.shape, maxval=0.1)
    fake_labels = tf.zeros_like(fake_output) + tf.random.uniform(shape=fake_output.shape, maxval=0.1)

    real_loss = cross_entropy(real_labels, real_output)
    fake_loss = cross_entropy(fake_labels, fake_output)

    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output, real_y):
    """
    Calcula a perda do gerador usando o erro médio quadrático (MSE).
    """
    real_y = tf.cast(real_y, tf.float32)
    return mse(fake_output, real_y)

learning_rate = 0.001
generator_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5, beta_2=0.999)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5, beta_2=0.999)

generator = get_generator_model()
discriminator = get_discriminator_model()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



## **3. Training The GAN**

So finally, we'll train our GAN on the dataset, we prepared earlier.


In [6]:
import tensorflow as tf
from tqdm import tqdm

gen_loss_metric = tf.keras.metrics.Mean(name="gen_loss")
disc_loss_metric = tf.keras.metrics.Mean(name="disc_loss")

early_stopping_patience = 10
best_gen_loss = float('inf')
patience_counter = 0

@tf.function
def train_step(input_x, real_y):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(input_x, training=True)

        real_output = discriminator(real_y, training=True)
        generated_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(generated_images, real_y)
        disc_loss = discriminator_loss(real_output, generated_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    gen_loss_metric.update_state(gen_loss)
    disc_loss_metric.update_state(disc_loss)

def train(dataset, epochs):
    """
    Loop principal de treinamento.
    """
    global best_gen_loss, patience_counter

    for epoch in range(epochs):
        gen_loss_metric.reset_state()
        disc_loss_metric.reset_state()

        print(f"\nEpoch {epoch+1}/{epochs}")
        progress_bar = tqdm(dataset, desc="Training", leave=True)

        for input_x, real_y in progress_bar:
            train_step(input_x, real_y)
            progress_bar.set_postfix({
                "Generator Loss": f"{gen_loss_metric.result():.4f}",
                "Discriminator Loss": f"{disc_loss_metric.result():.4f}"
            })

        print(f"Generator Loss: {gen_loss_metric.result():.4f}, Discriminator Loss: {disc_loss_metric.result():.4f}")

        current_gen_loss = gen_loss_metric.result()
        if current_gen_loss < best_gen_loss:
            best_gen_loss = current_gen_loss
            patience_counter = 0
            print("Validation improvement. Saving best model...\n")
            generator.save_weights("/content/drive/MyDrive/Mestrado/2024_2/Topicos_especiais_IA_ThiagoPX/best_generator.weights.h5")
            discriminator.save_weights("/content/drive/MyDrive/Mestrado/2024_2/Topicos_especiais_IA_ThiagoPX/best_discriminator.weights.h5")
        else:
            patience_counter += 1
            print(f"No improvement. Early stopping patience: {patience_counter}/{early_stopping_patience}")


        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered. Stopping training.")
            break





Run the cell below, to start the training

In [ ]:
train(dataset=dataset, epochs=500)


Epoch 1/500


Training: 100%|██████████| 9/9 [01:35<00:00, 10.60s/it, Generator Loss=0.2754, Discriminator Loss=1.8354]


Generator Loss: 0.2754, Discriminator Loss: 1.8354
Validation improvement. Saving best model...


Epoch 2/500


Training: 100%|██████████| 9/9 [00:16<00:00,  1.85s/it, Generator Loss=0.0367, Discriminator Loss=1.3886]


Generator Loss: 0.0367, Discriminator Loss: 1.3886
Validation improvement. Saving best model...


Epoch 3/500


...

Training: 100%|██████████| 9/9 [00:20<00:00,  2.27s/it, Generator Loss=0.0101, Discriminator Loss=1.3869]


Generator Loss: 0.0101, Discriminator Loss: 1.3869
Validation improvement. Saving best model...


Epoch 66/500


Training: 100%|██████████| 9/9 [00:16<00:00,  1.86s/it, Generator Loss=0.0102, Discriminator Loss=1.3863]


Generator Loss: 0.0102, Discriminator Loss: 1.3863
No improvement. Early stopping patience: 1/10

Epoch 67/500


Training:  44%|████▍     | 4/9 [00:08<00:10,  2.02s/it, Generator Loss=0.0101, Discriminator Loss=1.3871]


## **4. Results**

We plotted the input, output and the original images respectively, from a part of the dataset to find out the results.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

def display_results(generator, test_x, test_y, img_size=128, output_size=(1024, 1024), weights_path="/content/drive/MyDrive/Mestrado/2024_2/Topicos_especiais_IA_ThiagoPX/best_generator.weights.h5"):

    try:
        generator.load_weights(weights_path)
        print(f"Pesos carregados com sucesso de: {weights_path}")
    except Exception as e:
        print(f"Erro ao carregar pesos: {e}")
        return

    # Gerar previsões
    y_pred = generator(test_x).numpy()

    # Iterar sobre o conjunto de teste
    for i, (input_img, target_img, output_img) in enumerate(zip(test_x, test_y, y_pred)):
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        input_resized = Image.fromarray(input_img.reshape((128, 128)) * 255).resize(output_size)
        axes[0].imshow(input_resized, cmap='gray')
        axes[0].set_title("Imagem Cinza", fontsize=12)
        axes[0].axis("off")

        target_resized = Image.fromarray((target_img * 255).astype('uint8')).resize(output_size)
        axes[1].imshow(target_resized)
        axes[1].set_title("Saída Objetivo", fontsize=12)
        axes[1].axis("off")

        output_resized = Image.fromarray((output_img * 255).astype('uint8')).resize(output_size)
        axes[2].imshow(output_resized)
        axes[2].set_title("Imagem Colorida NN", fontsize=12)
        axes[2].axis("off")

        plt.tight_layout()
        plt.show()

display_results(generator, test_x, test_y, img_size=256, output_size=(1024, 1024), weights_path="best_generator.weights.h5")


Therefore, the overwhelming reslts depicts the power of GANs and the disruption which can be broght through them.